# THU THẬP DỮ LIỆU

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

df=pd.read_csv('full_tiki.csv')
df['index'] = range(0, len(df))
df.set_index(['index'])
df.head()

KeyboardInterrupt: 

In [ ]:
len(df)

In [ ]:
df.columns

In [ ]:
LE = LabelEncoder()
df['Type-Category'] = LE.fit_transform(df['Category'])

In [ ]:
df['Price'].isnull().sum() 

# TRÍCH XUẤT ĐẶC TRƯNG

## TRỰC QUAN HÓA DỮ LIỆU

In [ ]:
df['Category'].value_counts()

Có 9 loại sản phẩm được ưa chuộng 

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(x='Category', data=df, order = df['Category'].value_counts().index )
plt.xticks(rotation=45)

In [ ]:
max(df['Price'])

In [ ]:
min(df['Price'])

In [ ]:
data = df.copy()
data_100nghin = data.loc[data['List-Price'] <= 100000]
data_100nghin['Category'].value_counts().plot(kind = 'barh')

In [ ]:
data_500nghin = data.loc[ (100000 < data['List-Price']) & ( data['List-Price'] <= 500000 )]
data_500nghin['Category'].value_counts().plot(kind = 'barh')

In [ ]:
data_1trieu = data.loc[ (500000 < data['List-Price']) & ( data['List-Price'] <= 1000000 )]
data_1trieu['Category'].value_counts().plot(kind = 'barh')

In [ ]:
data_5trieu = data.loc[ (1000000 < data['List-Price']) & ( data['List-Price'] <= 5000000 )]
data_5trieu['Category'].value_counts().plot(kind = 'barh')

In [ ]:
data_10trieu = data.loc[ (5000000 < data['List-Price']) & ( data['List-Price'] <= 10000000 )]
data_10trieu['Category'].value_counts().plot(kind = 'barh')

In [ ]:
data_L10trieu = data.loc[ (10000000 < data['List-Price'])]
data_L10trieu['Category'].value_counts().plot(kind = 'barh')

In [ ]:
df['Rating Average'].hist()

In [ ]:
df['Price'].hist()

In [ ]:
df.boxplot(column = ['Rating Average','Price', 'Discount Rate'])

In [ ]:
df.boxplot(column = 'Rating Average')

In [ ]:
df.boxplot(column = 'Price')

In [ ]:
df[['Rating Average','Price', 'Discount Rate']].corr

In [ ]:
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(df[['Rating Average','Price','Discount Rate']].corr(),vmax=.5, center=0, cmap=cmap,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
pd.pivot_table(df, index = 'Category', values = 'Price') # tính mức giá trung bình của từng loại sản phẩm 

In [ ]:
pd.options.display.max_rows
pd.set_option('display.max_rows', None) # cho phép hiển thị tất cả các dòng

Outliers

In [ ]:
data.loc[data['Price'] >= 1969000,'Price'] = 1969000
data.loc[data['Total Review'] >= 734,'Total Review']= 734
data.loc[data['Discount Rate'] >=33,'Discount Rate']= 33

# data.loc[data['Price'] <= 10000,'Price'] = 10000
# data.loc[data['Total Review'] <= 20,'Total Review']= 20
# data.loc[data['Discount Rate'] <=5,'Discount Rate']= 5

In [ ]:
last_data = data.loc[data['Rating Average'] >= 3]

In [ ]:
pd.pivot_table(last_data, index = ['Category','Rating Average'], values = 'Price') 

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'
last_data.loc[last_data['Rating Average'] < 4 ,'Rating Average']=3
last_data.loc[ (last_data['Rating Average'] >= 4)
              & (last_data['Rating Average'] < 4.8 ),'Rating Average']=4   
# last_data.loc[ (last_data['Rating Average'] >= 4.4)
#               & (last_data['Rating Average'] < 4.8 ),'Rating Average']=4.5
last_data.loc[last_data['Rating Average'] >= 4.8 ,'Rating Average']=5

In [ ]:
last_data['Rating Average'].value_counts()

# MÔ HÌNH HÓA DỮ LIỆU

## CHỌN MÔ HÌNH

### HỒI QUY TUYẾN TÍNH

In [ ]:
last_data['New-Rate'] = last_data['Rating Average'].apply(lambda x: x * 10 )
# mô hình hồi quy tuyến tính yêu cầu trả về số nguyên

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(last_data[['Price','Discount Rate','Total Review','Type-Category']],
                                               last_data['New-Rate'],
                                               test_size=0.3, random_state=5) # train here

In [ ]:
### Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

classifier=LogisticRegression(max_iter=1000)
classifier.fit(X_train,y_train)
y_pred=classifier.predict(X_test)

print("Accuracy_score: {}".format(accuracy_score(y_test,y_pred)))

In [ ]:
def Predict_Score(new_data):
    list_accuracy_score = []
    for i in range (10):
        X_train,X_test,y_train,y_test=train_test_split(new_data[['Price','Discount Rate','Total Review','Type-Category']], 
                                                       new_data['New-Rate'], 
                                                       test_size=0.3, random_state=i)
        classifier=LogisticRegression()
        classifier.fit(X_train,y_train)
        y_pred=classifier.predict(X_test)
        list_accuracy_score.append(accuracy_score(y_test,y_pred))
    return np.mean(list_accuracy_score)

In [ ]:
Predict_Score(last_data)

In [ ]:
new_data = data.loc[data['Rating Average'] < 3]

In [ ]:
predict_data = new_data[['Price','Discount Rate','Total Review','Type-Category']]
predict_data.head()

In [ ]:
y_pred = classifier.predict(predict_data)

In [ ]:
y_pred / 10 

In [ ]:
from sklearn import svm

last_data = data.loc[data['Rating Average'] >= 3]
last_data.loc[last_data['Rating Average'] < 3.4  ,'Rating Average']=3
last_data.loc[ (last_data['Rating Average'] >= 3.4)
              & (last_data['Rating Average'] < 3.8 ),'Rating Average']=3.5
last_data.loc[ (last_data['Rating Average'] >= 3.8)
              & (last_data['Rating Average'] < 4.4 ),'Rating Average']=4
last_data.loc[ (last_data['Rating Average'] >= 4.4)
              & (last_data['Rating Average'] < 4.8 ),'Rating Average']=4.5
# last_data.loc[last_data['Rating Average'] >= 4.8 ,'Rating Average'] = 5
last_data['New-Rate'] = last_data['Rating Average'].apply(lambda x: x * 10 )
X_train,X_test,y_train,y_test=train_test_split(last_data[['Price','Discount Rate','Total Review','Type-Category']],
                                               last_data['New-Rate'],
                                               test_size=0.4, random_state=5) # train here
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)


In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))